# Projeto Banco de Dados
# Ocorrências Policiais de São Paulo/SP

# Objetivo
Com base em informações de Ocorrências Policiais do Estado de São Paulo, entre os anos de 2002 e 2021, pretende-se neste projeto avaliar alguns questionamentos e encontrar possíveis soluções com base no tema da segurança pública.

Para construção deste projeto, o material seguirá as seguintes etapas:

**1. Detalhamento:**
  Nesta etapa consta as características do dataset escolhido;

**2. Importação de Dados:** 
  Dados serão importados e preparados;

**3. Criação do Banco de Dados:** 
  Criando o banco e as tabelas que serão necessárias;

**4. Criação de Tabela Ouro:**
   Criando a tabela final que será utilizada nas análises;

**5. Análises:**
  Consultas com análises a respeito dos dados obtidos, como:

-   5 anos com maiores incidentes de homicidio;
-   Top 10 cidades em registros de homicídio nos últimos 10 anos (2011-2021);
-   Top 5 cidades em registros de Roubo e Furto na última década (2010 - 2019);
-   Histórico da última década (2010 - 2019) de Roubo e Furtos na cidade de São Paulo;
-  Top 5 anos em quantidade de ocorrências policiais no estado;
     

**6. Autoavaliação.**

# 1. Detalhamento

## 1.1 - Origem dos Dados
Dados públicos fornecidos pela Secretaria de Segurança Pública do Estado de São Paulo entre os anos de 2002 a 2021.

## 1.2 - Catálogo de Dados

**1.2.1 Tabela de Ocorrências:**

Tabela com quantidade de ocorrências policiais registradas em todos os municípios do estado de São Paulo.

![Catalogo Ocorrencias](https://raw.githubusercontent.com/pauloricardofagundes/MVP-Engenharia_Dados/refs/heads/main/Catalogo_dados_MVP1.png)

**1.2.2 Tabela de Municípios:**

Tabela que será utilizada apenas para localizar o nome do município, de acordo com a chave primária "id_municipio" que consta em ambas tabelas.

_(Serão carregados para criação do banco de dados apenas os 2 campos necessários para identificação do município)_

![Catalogo Municipios](https://raw.githubusercontent.com/pauloricardofagundes/MVP-Engenharia_Dados/refs/heads/main/Catalogo_dados_MVP2.png)

# 2. Importação dos dados para criação do Database

In [0]:
## Importando Bibliotecas
from pyspark import SparkFiles
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

In [0]:
# Base de dados com registro de ocorrências
url_1 = "https://raw.githubusercontent.com/pauloricardofagundes/MVP-Engenharia_Dados/refs/heads/main/br_sp_gov_ssp_ocorrencias_registradas.csv"  

# Base de identificação de municícipios, para join com a base de ocorrências
url_2 = "https://raw.githubusercontent.com/pauloricardofagundes/MVP-Engenharia_Dados/refs/heads/main/br_bd_diretorios_brasil_municipio.csv"  

# Leitura dos datasets
spark.sparkContext.addFile(url_1)
df_bd_ocorrencias = spark.read.csv(
    "file://" + SparkFiles.get("br_sp_gov_ssp_ocorrencias_registradas.csv"),
    header=True,
    inferSchema=False,
)

spark.sparkContext.addFile(url_2)
df_bd_municipios = spark.read.csv(
    "file://" + SparkFiles.get("br_bd_diretorios_brasil_municipio.csv"),
    header=True,
    inferSchema=False,
)

# Convertendo campos numéricos de texto para número inteiro
Campos_Num = [  "ano",
                "homicidio_doloso",
                "numero_de_vitimas_em_homicidio_doloso",
                "homicidio_doloso_por_acidente_de_transito",
                "numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito",
                "homicidio_culposo_por_acidente_de_transito",
                "homicidio_culposo_outros",
                "tentativa_de_homicidio",
                "lesao_corporal_seguida_de_morte",
                "lesao_corporal_dolosa",
                "lesao_corporal_culposa_por_acidente_de_transito",
                "lesao_corporal_culposa_outras",
                "latrocinio",
                "numero_de_vitimas_em_latrocinio",
                "total_de_estupro",
                "estupro",
                "estupro_de_vulneravel",
                "total_de_roubo_outros",
                "roubo_outros",
                "roubo_de_veiculo",
                "roubo_a_banco",
                "roubo_de_carga",
                "furto_outros",
                "furto_de_veiculo"
                ]

for x in Campos_Num: 
    df_bd_ocorrencias = df_bd_ocorrencias.withColumn(x, df_bd_ocorrencias[x].cast(IntegerType()))

# Selecionar colunas específicas da tabela de município
colunas_desejadas = ["id_municipio", "nome"]
df_bd_municipios = df_bd_municipios.select(colunas_desejadas)


# 3. Criação do Banco de Dados

In [0]:
%sql
-- Deletando o banco de dados (caso exista)

DROP DATABASE Db_Ocorrencia CASCADE; 


In [0]:
%sql
-- Criando o novo banco de dados

CREATE DATABASE Db_Ocorrencia; 

In [0]:
## Criando as Tabelas "ocorrencias" e "municipios"

df_bd_ocorrencias.write.mode("overwrite").saveAsTable("Db_Ocorrencia.ocorrencias")
df_bd_municipios.write.mode("overwrite").saveAsTable("Db_Ocorrencia.municipios")

In [0]:
%sql 
-- Verificando a tabela criada "ocorrencias"

SELECT * 
FROM Db_Ocorrencia.ocorrencias
LIMIT 5

ano,mes,id_municipio,regiao_ssp,homicidio_doloso,numero_de_vitimas_em_homicidio_doloso,homicidio_doloso_por_acidente_de_transito,numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito,homicidio_culposo_por_acidente_de_transito,homicidio_culposo_outros,tentativa_de_homicidio,lesao_corporal_seguida_de_morte,lesao_corporal_dolosa,lesao_corporal_culposa_por_acidente_de_transito,lesao_corporal_culposa_outras,latrocinio,numero_de_vitimas_em_latrocinio,total_de_estupro,estupro,estupro_de_vulneravel,total_de_roubo_outros,roubo_outros,roubo_de_veiculo,roubo_a_banco,roubo_de_carga,furto_outros,furto_de_veiculo
2002,1,3500105,Presidente Prudente,0,0,0,0,0,0,0,0,24,13,2,0,0,0,null,null,0,null,0,0,0,21,0
2002,2,3500105,Presidente Prudente,0,0,0,0,0,0,0,0,24,15,2,0,0,0,null,null,0,null,0,0,0,32,0
2002,3,3500105,Presidente Prudente,0,0,0,0,0,0,1,0,21,13,5,0,0,0,null,null,0,null,0,0,0,36,0
2002,4,3500105,Presidente Prudente,0,0,0,0,0,0,1,0,36,13,1,0,0,0,null,null,1,null,0,0,0,45,0
2002,5,3500105,Presidente Prudente,0,0,0,0,0,0,0,0,27,12,1,0,0,0,null,null,1,null,0,0,0,35,0


In [0]:
%sql
-- Verificando a tabela criada "municipios"

SELECT * 
FROM Db_Ocorrencia.municipios
LIMIT 5

id_municipio,nome
1100338,Nova Mamoré
1100205,Porto Velho
1101104,Itapuã do Oeste
1100809,Candeias do Jamari
1100940,Cujubim


# 4. Junção de tabelas para criação de tabela flat final

In [0]:
%sql
-- Validando quantidade de registros únicos de municípios

SELECT
Id_municipio,
nome,
COUNT(*) as contagem
FROM Db_Ocorrencia.municipios
GROUP BY id_municipio, nome
HAVING COUNT(*) > 1
ORDER BY contagem DESC

Id_municipio,nome,contagem


In [0]:
%sql
-- Junção das tabelas "ocorrencia" e "municipio", criando a tabela ouro final

CREATE TABLE Db_Ocorrencia.Geral AS 
SELECT 
O.* ,
M.nome
FROM Db_Ocorrencia.ocorrencias AS O
 
LEFT JOIN 
 (SELECT DISTINCT
 Id_municipio,
 nome
 From Db_Ocorrencia.municipios ) AS M
ON O.id_municipio = M.id_municipio

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Testando a tabela final

SELECT * FROM db_ocorrencia.geral
LIMIT 5

ano,mes,id_municipio,regiao_ssp,homicidio_doloso,numero_de_vitimas_em_homicidio_doloso,homicidio_doloso_por_acidente_de_transito,numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito,homicidio_culposo_por_acidente_de_transito,homicidio_culposo_outros,tentativa_de_homicidio,lesao_corporal_seguida_de_morte,lesao_corporal_dolosa,lesao_corporal_culposa_por_acidente_de_transito,lesao_corporal_culposa_outras,latrocinio,numero_de_vitimas_em_latrocinio,total_de_estupro,estupro,estupro_de_vulneravel,total_de_roubo_outros,roubo_outros,roubo_de_veiculo,roubo_a_banco,roubo_de_carga,furto_outros,furto_de_veiculo,nome
2002,1,3500105,Presidente Prudente,0,0,0,0,0,0,0,0,24,13,2,0,0,0,null,null,0,null,0,0,0,21,0,Adamantina
2002,2,3500105,Presidente Prudente,0,0,0,0,0,0,0,0,24,15,2,0,0,0,null,null,0,null,0,0,0,32,0,Adamantina
2002,3,3500105,Presidente Prudente,0,0,0,0,0,0,1,0,21,13,5,0,0,0,null,null,0,null,0,0,0,36,0,Adamantina
2002,4,3500105,Presidente Prudente,0,0,0,0,0,0,1,0,36,13,1,0,0,0,null,null,1,null,0,0,0,45,0,Adamantina
2002,5,3500105,Presidente Prudente,0,0,0,0,0,0,0,0,27,12,1,0,0,0,null,null,1,null,0,0,0,35,0,Adamantina


# 5. Análise

## 5.1 - Qualidade de Dados

> Analisando os dados gerais, foram encontrados problemas em alguns campos de armazenamento quantitativo, que não mantêm uma consistência ao longo da série histórica, sugerindo que alguns campos foram criados para complementar a tabela de dados final.

## 5.2 - Solução de Problemas

### 5.2.1 - 5 anos com maiores incidentes de homicidio no Estado de São Paulo
> É possível observar que os 5 anos com maior volume de homicídios no estado do período avaliado, foram os primeiros anos dessa série, em order cronológica crescente de 2002 a 2006.

In [0]:
%sql
--// Selecionando as colunas que contém dados de homicídio, somando elas, agrupando por ano e ordenando pelos 5 maiores resultados

SELECT
ano as Ano,
Sum(COALESCE(homicidio_doloso, 0)) as Hom_Doloso,
Sum(COALESCE(homicidio_doloso_por_acidente_de_transito, 0)) as Hom_Dol_AT,
Sum(COALESCE(homicidio_culposo_outros, 0)) as Hom_Cul,
Sum(COALESCE(homicidio_culposo_por_acidente_de_transito, 0)) as Hom_Cul_AT,
Sum(COALESCE(homicidio_doloso, 0)) +
  Sum(COALESCE(homicidio_doloso_por_acidente_de_transito, 0)) +
  Sum(COALESCE(homicidio_culposo_outros, 0)) +
  Sum(COALESCE(homicidio_culposo_por_acidente_de_transito, 0))  as Total,

Round(((
  Sum(COALESCE(homicidio_doloso, 0)) +
  Sum(COALESCE(homicidio_doloso_por_acidente_de_transito, 0)) + 
  Sum(COALESCE(homicidio_culposo_outros, 0)) + 
  Sum(COALESCE(homicidio_culposo_por_acidente_de_transito, 0))) 
  * 100.0 / 
  (SELECT (
    Sum(COALESCE(homicidio_doloso, 0)) + 
    Sum(COALESCE(homicidio_doloso_por_acidente_de_transito, 0)) + 
    Sum(COALESCE(homicidio_culposo_outros, 0)) + 
    Sum(COALESCE(homicidio_culposo_por_acidente_de_transito, 0)))
    FROM Db_Ocorrencia.geral)),2) AS `% Perc`

FROM Db_Ocorrencia.geral
GROUP BY Ano
ORDER BY Total DESC
LIMIT 5

Ano,Hom_Doloso,Hom_Dol_AT,Hom_Cul,Hom_Cul_AT,Total,% Perc
2002,11846,0,333,4740,16919,8.96
2003,10943,0,377,4601,15921,8.43
2004,8743,0,230,4609,13582,7.19
2005,7076,0,259,4708,12043,6.37
2006,6050,0,256,4479,10785,5.71


___

### 5.2.2 - Top 10 cidades em registros de homicídio nos últimos 10 anos (2011-2021)
> Durante o período de 2011 a 2021,  cidade com maior número de homicídios no estado de São Paulo foi a capital Paulista, com 8% de todo o volume de ocorrências, em comparação com os demais municípios do estado.

In [0]:
%sql
--// Selecionando as colunas que contém dados de homicídio, somando elas, agrupando por município e ordenando pelos 10 maiores resultados

SELECT
nome as Municipio,
Sum(COALESCE(homicidio_doloso,0)) as Hom_Doloso,
Sum(COALESCE(homicidio_doloso_por_acidente_de_transito,0)) as Hom_Dol_AT,
Sum(COALESCE(homicidio_culposo_outros,0)) as Hom_Cul,
Sum(COALESCE(homicidio_culposo_por_acidente_de_transito,0)) as Hom_Cul_AT,
Sum(COALESCE(homicidio_doloso,0)) +
  Sum(COALESCE(homicidio_doloso_por_acidente_de_transito,0)) +
  Sum(COALESCE(homicidio_culposo_outros,0)) +
  Sum(COALESCE(homicidio_culposo_por_acidente_de_transito,0))  as Total,

Round(((
  Sum(COALESCE(homicidio_doloso,0)) +
  Sum(COALESCE(homicidio_doloso_por_acidente_de_transito,0)) + 
  Sum(COALESCE(homicidio_culposo_outros,0)) + 
  Sum(COALESCE(homicidio_culposo_por_acidente_de_transito,0))) 
  * 100.0 / 
  (SELECT (
    Sum(COALESCE(homicidio_doloso,0)) + 
    Sum(COALESCE(homicidio_doloso_por_acidente_de_transito,0)) + 
    Sum(COALESCE(homicidio_culposo_outros,0)) + 
    Sum(COALESCE(homicidio_culposo_por_acidente_de_transito,0))) 
    FROM Db_Ocorrencia.geral)),2) AS `% Perc`

FROM Db_Ocorrencia.geral
WHERE Ano BETWEEN 2011 and 2021
GROUP BY Municipio
ORDER BY Total DESC
LIMIT 10

Municipio,Hom_Doloso,Hom_Dol_AT,Hom_Cul,Hom_Cul_AT,Total,% Perc
São Paulo,9610,135,485,4998,15228,8.06
Campinas,1411,5,57,1201,2674,1.42
Guarulhos,1401,11,56,920,2388,1.26
Sorocaba,602,6,17,596,1221,0.65
São Bernardo do Campo,597,14,35,535,1181,0.63
São José dos Campos,572,2,21,577,1172,0.62
Ribeirão Preto,539,11,16,538,1104,0.58
Osasco,668,4,21,328,1021,0.54
Jundiaí,235,6,40,587,868,0.46
Santo André,538,6,23,244,811,0.43


___

### 5.2.3 - Top 5 cidades em registros de Roubo e Furto entre 2010 e 2019
> Avaliando a década de 2010, a cidade de São Paulo também aparece com o maior volume de ocorrências de roubos e furtos no estado, representando 21% do total destes tipos de ocorrências.

In [0]:
%sql
--// Selecionando as colunas que contém dados de roubo e furto entre 2010 e 2019, somando elas, agrupando por município e ordenando pelos 5 maiores resultados

SELECT
nome as Municipio,
Sum(COALESCE(roubo_outros,0)) as r_outro,
Sum(COALESCE(roubo_a_banco,0)) as r_banco,
Sum(COALESCE(roubo_de_carga,0)) as r_carga,
Sum(COALESCE(furto_outros,0)) as f_outro,
Sum(COALESCE(furto_de_veiculo,0)) as f_veiculo,

Sum(COALESCE(roubo_outros,0)) + 
  Sum(COALESCE(roubo_a_banco,0)) + 
  sum(COALESCE(roubo_de_carga,0)) + 
  sum(COALESCE(furto_outros,0)) + 
  sum(COALESCE(furto_de_veiculo,0)) as Total,

Round(((
  Sum(COALESCE(roubo_outros,0)) +
  Sum(COALESCE(roubo_a_banco,0)) + 
  Sum(COALESCE(roubo_de_carga,0)) + 
  Sum(COALESCE(furto_outros,0)) +
  Sum(COALESCE(furto_de_veiculo,0))) 
  * 100.0 / 
  (SELECT (
    Sum(COALESCE(roubo_outros,0)) + 
    Sum(COALESCE(roubo_a_banco,0)) + 
    Sum(COALESCE(roubo_de_carga,0)) + 
    Sum(COALESCE(furto_outros,0)) +
    Sum(COALESCE(furto_de_veiculo,0))) 
    FROM Db_Ocorrencia.geral)),2) AS `% Perc`

FROM Db_Ocorrencia.geral
WHERE Ano BETWEEN 2010 and 2019
GROUP BY Municipio
ORDER BY Total DESC
LIMIT 5

Municipio,r_outro,r_banco,r_carga,f_outro,f_veiculo,Total,% Perc
São Paulo,416519,817,47421,1942482,435550,2842789,21.16
Campinas,22243,52,2473,175257,42721,242746,1.81
Guarulhos,20848,29,3901,104217,29651,158646,1.18
Ribeirão Preto,9815,6,332,108422,23205,141780,1.06
Santo André,22464,36,1033,75385,36394,135312,1.01


___

### 5.2.4 - Histórico da última década (2010-2019) de Roubos e Furtos na cidade de São Paulo
> Os últimos 3 anos deste período são os que apresentam maior volume deste tipo de ocorrência, porém podemos ter uma ruído nos dados aqui, uma vez que é somente à partir de 2017 que começam a aparecer registros de "roubo_outros".

In [0]:
%sql
--// Selecionando as colunas que contém dados de roubo e furto entre 2010 e 2019 da cidade de São Paulo, somando elas, agrupando por ano e ordenando pelos 10 maiores resultados

SELECT
ano as Ano,
nome as Municipio,
Sum(COALESCE(roubo_outros,0)) as r_outro,
Sum(COALESCE(roubo_a_banco,0)) as r_banco,
Sum(COALESCE(roubo_de_carga,0)) as r_carga,
Sum(COALESCE(furto_outros,0)) as f_outro,
Sum(COALESCE(furto_de_veiculo,0)) as f_veiculo,

Sum(COALESCE(roubo_outros,0)) + 
  Sum(COALESCE(roubo_a_banco,0)) + 
  sum(COALESCE(roubo_de_carga,0)) + 
  sum(COALESCE(furto_outros,0)) + 
  sum(COALESCE(furto_de_veiculo,0)) as Total,

Round(((
  Sum(COALESCE(roubo_outros,0)) +
  Sum(COALESCE(roubo_a_banco,0)) + 
  Sum(COALESCE(roubo_de_carga,0)) + 
  Sum(COALESCE(furto_outros,0)) +
  Sum(COALESCE(furto_de_veiculo,0))) 
  * 100.0 / 
  (SELECT (
    Sum(COALESCE(roubo_outros,0)) + 
    Sum(COALESCE(roubo_a_banco,0)) + 
    Sum(COALESCE(roubo_de_carga,0)) + 
    Sum(COALESCE(furto_outros,0)) +
    Sum(COALESCE(furto_de_veiculo,0))) 
    FROM Db_Ocorrencia.geral)),2) AS `% Perc`

FROM Db_Ocorrencia.geral
WHERE Ano BETWEEN 2010 and 2019
  AND nome = 'São Paulo'
GROUP BY ano, nome
ORDER BY Ano ASC

Ano,Municipio,r_outro,r_banco,r_carga,f_outro,f_veiculo,Total,% Perc
2010,São Paulo,0,141,4475,170181,42930,217727,1.62
2011,São Paulo,0,149,4258,198252,42778,245437,1.83
2012,São Paulo,0,115,4353,199428,42925,246821,1.84
2013,São Paulo,0,122,4581,201305,48477,254485,1.89
2014,São Paulo,0,86,5092,190717,49703,245598,1.83
2015,São Paulo,0,81,5066,177199,43796,226142,1.68
2016,São Paulo,0,43,5866,183350,44668,233927,1.74
2017,São Paulo,146818,41,5747,193686,42547,388839,2.89
2018,São Paulo,132332,26,4231,201798,40395,378782,2.82
2019,São Paulo,137369,13,3752,226566,37331,405031,3.01


___

### 5.2.5 - Top 5 anos em quantidade de ocorrências policiais no estado.
> Dentre os 5 anos com maiores ocorrências de toda a base de dados, os 3 principais foram os anos de 2017, 2018 e 2019 respectivamente.

In [0]:
%sql
--// Selecionando todas as ocorrências do período (2002 - 2021), somando elas, agrupando por ano e ordenando pelos 5 maiores resultados

SELECT
ano as Ano,
Sum(COALESCE(estupro,0)) + 
  Sum(COALESCE(estupro_de_vulneravel,0)) AS TT_Estup,
Sum(COALESCE(furto_de_veiculo,0)) + 
  Sum(COALESCE(furto_outros,0)) AS TT_Furto,
Sum(COALESCE(homicidio_culposo_outros,0)) + 
  Sum(COALESCE(homicidio_doloso,0)) + 
  Sum(COALESCE(homicidio_doloso_por_acidente_de_transito,0)) AS TT_Homic,
Sum(COALESCE(latrocinio,0)) AS Latroc,
Sum(COALESCE(lesao_corporal_culposa_outras,0)) +
  Sum(COALESCE(lesao_corporal_culposa_por_acidente_de_transito,0)) +
  Sum(COALESCE(lesao_corporal_dolosa,0)) +
  Sum(COALESCE(lesao_corporal_seguida_de_morte,0)) AS TT_LesCorp,
Sum(COALESCE(roubo_a_banco,0)) +
  Sum(COALESCE(roubo_de_carga,0)) +
  Sum(COALESCE(roubo_de_veiculo,0)) +
  Sum(COALESCE(roubo_outros,0)) AS TT_Roubo,
Sum(COALESCE(tentativa_de_homicidio,0)) AS Tent_Hm,


Sum(COALESCE(estupro,0))+
Sum(COALESCE(estupro_de_vulneravel,0))+
Sum(COALESCE(furto_de_veiculo,0))+
Sum(COALESCE(furto_outros,0))+
Sum(COALESCE(homicidio_culposo_outros,0))+
Sum(COALESCE(homicidio_doloso,0))+
Sum(COALESCE(homicidio_doloso_por_acidente_de_transito,0))+
Sum(COALESCE(latrocinio,0))+
Sum(COALESCE(lesao_corporal_culposa_outras,0))+
Sum(COALESCE(lesao_corporal_culposa_por_acidente_de_transito,0))+
Sum(COALESCE(lesao_corporal_dolosa,0))+
Sum(COALESCE(lesao_corporal_seguida_de_morte,0))+
Sum(COALESCE(roubo_a_banco,0))+
Sum(COALESCE(roubo_de_carga,0))+
Sum(COALESCE(roubo_de_veiculo,0))+
Sum(COALESCE(roubo_outros,0))+
Sum(COALESCE(tentativa_de_homicidio,0))
as Total,

Round(((
  Sum(COALESCE(estupro,0))+
  Sum(COALESCE(estupro_de_vulneravel,0))+
  Sum(COALESCE(furto_de_veiculo,0))+
  Sum(COALESCE(furto_outros,0))+
  Sum(COALESCE(homicidio_culposo_outros,0))+
  Sum(COALESCE(homicidio_doloso,0))+
  Sum(COALESCE(homicidio_doloso_por_acidente_de_transito,0))+
  Sum(COALESCE(latrocinio,0))+
  Sum(COALESCE(lesao_corporal_culposa_outras,0))+
  Sum(COALESCE(lesao_corporal_culposa_por_acidente_de_transito,0))+
  Sum(COALESCE(lesao_corporal_dolosa,0))+
  Sum(COALESCE(lesao_corporal_seguida_de_morte,0))+
  Sum(COALESCE(roubo_a_banco,0))+
  Sum(COALESCE(roubo_de_carga,0))+
  Sum(COALESCE(roubo_de_veiculo,0))+
  Sum(COALESCE(roubo_outros,0))+
  Sum(COALESCE(tentativa_de_homicidio,0))) 
  * 100.0 / 
  (SELECT (
    Sum(COALESCE(estupro,0))+
    Sum(COALESCE(estupro_de_vulneravel,0))+
    Sum(COALESCE(furto_de_veiculo,0))+
    Sum(COALESCE(furto_outros,0))+
    Sum(COALESCE(homicidio_culposo_outros,0))+
    Sum(COALESCE(homicidio_doloso,0))+
    Sum(COALESCE(homicidio_doloso_por_acidente_de_transito,0))+
    Sum(COALESCE(latrocinio,0))+
    Sum(COALESCE(lesao_corporal_culposa_outras,0))+
    Sum(COALESCE(lesao_corporal_culposa_por_acidente_de_transito,0))+
    Sum(COALESCE(lesao_corporal_dolosa,0))+
    Sum(COALESCE(lesao_corporal_seguida_de_morte,0))+
    Sum(COALESCE(roubo_a_banco,0))+
    Sum(COALESCE(roubo_de_carga,0))+
    Sum(COALESCE(roubo_de_veiculo,0))+
    Sum(COALESCE(roubo_outros,0))+
    Sum(COALESCE(tentativa_de_homicidio,0))
) 
    FROM Db_Ocorrencia.geral)),2) AS `% Perc`

FROM Db_Ocorrencia.geral
GROUP BY ano
ORDER BY Total DESC
LIMIT 5

Ano,TT_Estup,TT_Furto,TT_Homic,Latroc,TT_LesCorp,TT_Roubo,Tent_Hm,Total,% Perc
2017,11072,619394,3500,331,228495,370519,3804,1237115,5.96
2018,11943,604177,3137,270,207243,322084,3484,1152338,5.55
2019,12371,612737,2977,192,208390,301909,3364,1141940,5.50
2005,0,682491,7335,315,347269,86532,8188,1132130,5.46
2004,0,673448,8973,390,325176,86076,9367,1103430,5.32


# 6 - Autoavaliação
> Avaliando o trabalho como um todo, no início tive maior dificuldade em compreender o modo em como estruturar todos os requisitos que seriam avaliados.
> 
> Localizei uma base de dados que avaliei interessante para este trabalho e iniciei as etapas de importação, tratamento dos dados e criação do banco de dados e à partir daí elaborando as possíveis análises que seriam feitas dos dados.
> 
> No database geral, não encontrei problemas nos dados que devessem ser expurgados e/ou substituídos. Foi necessária apenas uma conversão de valores nulos ao fazer somas, pois alguns campos possuíam apenas parte do histórico.
>
> A conclusão geral é de que foi possível trabalhar com os dados obtidos, fazer as análises que julguei serem pertinentes de acordo com o tema escolhido.
>
> Avaliando o database obtido, entendo que poderia ter sido melhor estruturado, apenas com os dados das ocorrências, sem campos totalizadores incompletos em meio aos demais, o que aumenta o armazenamento e torna esse tipo de campo inútil para análises.